In [1]:
from data import MNQ, MES, CL, GC, ZC, ZB

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [5]:
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display

def session_returns(data, start_time, end_time):
    session_data = data.between_time(start_time, end_time)
    session_open = session_data['Open'].iloc[0] if not session_data.empty else None
    session_close = session_data['Close'].iloc[-1] if not session_data.empty else None
    if session_open and session_close:
        abs_change = session_close - session_open
        pct_change = ((session_close - session_open) / session_open) * 100
        return f"{abs_change:.2f} ({pct_change:.2f}%)"
    else:
        return None

sessions = {
    "Tokyo Session": ("19:00", "04:00"),
    "London Session": ("03:00", "12:00"),
    "New York Session": ("7:00", "16:00")
}

tickers_data = {
    "MNQ": MNQ,
    "MES": MES,
    "CL": CL,
    "GC": GC,
    "ZC": ZC,
    "ZB": ZB
}

yesterday = (datetime.now() - timedelta(days=1)).date()
combined_data = []

for ticker, data in tickers_data.items():
    daily_data = data[data.index.date == yesterday]
    daily_returns = {"Instrument": ticker, "Date": yesterday}
    for session_name, (start_time, end_time) in sessions.items():
        daily_returns[session_name] = session_returns(daily_data, start_time, end_time)
    combined_data.append(daily_returns)

combined_df = pd.DataFrame(combined_data)
combined_df.set_index("Instrument", inplace=True)

def highlight_returns(val):
    if val is not None and '(' in val:
        pct_value = float(val.split('(')[-1].replace('%', '').replace(')', ''))
        color = 'green' if pct_value > 0 else 'red' if pct_value < 0 else 'black'
        return f'color: {color}'
    return 'color: black'

styled_df = combined_df.style.applymap(highlight_returns, subset=["Tokyo Session", "London Session", "New York Session"])
display(styled_df)


,Date,Tokyo Session,London Session,New York Session
Instrument,,,,
MNQ,2024-10-31,-326.75 (-1.60%),-291.00 (-1.43%),-273.00 (-1.34%)
MES,2024-10-31,-78.25 (-1.34%),-54.75 (-0.94%),-59.75 (-1.03%)
CL,2024-10-31,1.59 (2.30%),0.37 (0.54%),1.68 (2.44%)
GC,2024-10-31,-34.60 (-1.24%),-40.50 (-1.45%),-35.10 (-1.26%)
ZC,2024-10-31,2.00 (0.49%),1.50 (0.37%),0.75 (0.18%)
ZB,2024-10-31,0.31 (0.27%),-0.28 (-0.24%),-0.06 (-0.05%)
